In [1]:
import pandas as pd
import ast
import json

In [2]:
#Lectura desde archivo parquet. 
# No se encuentrasn presentes en el repositorio los originales en csv por el peso de los mismos. 
# #En su lugar fueron cargados copias en parquet para que pueda ejecutarse el código.
df_credits = pd.read_parquet(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\credits.parquet")

In [3]:
df_credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [4]:
# Conversión de cadenas a listas de diccionarios
def safe_eval(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return []
    return x if isinstance(x, list) else []

# Conversión de las columnas 'cast' y 'crew' a listas
df_credits['cast'] = df_credits['cast'].apply(safe_eval)
df_credits['crew'] = df_credits['crew'].apply(safe_eval)

# Creación de listas para almacenar los datos expandidos
cast_data = []
crew_data = []

# Iteración sobre las filas de df_credits
for index, row in df_credits.iterrows():
    movie_id = row['id']
    
    # Procesar cast
    for cast_member in row['cast']:
        cast_member['original_index'] = index
        cast_member['movie_id'] = movie_id
        cast_data.append(cast_member)
    
    # Procesar crew
    for crew_member in row['crew']:
        crew_member['original_index'] = index
        crew_member['movie_id'] = movie_id
        crew_data.append(crew_member)

# Creación de los DataFrames
cast_df = pd.DataFrame(cast_data)
crew_df = pd.DataFrame(crew_data)

print("Cast DataFrame Info:")
print(cast_df.info())

print("\nCrew DataFrame Info:")
print(crew_df.info())

# Verificación de si todos los IDs de película están presentes
print("\nNúmero de filas en el DataFrame original:", len(df_credits))
print("Número de IDs únicos en el DataFrame original:", df_credits['id'].nunique())
print("Número de filas en cast_df:", len(cast_df))
print("Número de IDs únicos en cast_df:", cast_df['movie_id'].nunique())
print("Número de filas en crew_df:", len(crew_df))
print("Número de IDs únicos en crew_df:", crew_df['movie_id'].nunique())

# Verificación de las películas sin datos de cast o crew
movies_without_cast = set(df_credits['id']) - set(cast_df['movie_id'])
movies_without_crew = set(df_credits['id']) - set(crew_df['movie_id'])

print("\nNúmero de películas sin datos de cast:", len(movies_without_cast))
print("Número de películas sin datos de crew:", len(movies_without_crew))

# Verificación de duplicados
print("\nNúmero de IDs duplicados en el DataFrame original:", df_credits['id'].duplicated().sum())
print("Número de IDs duplicados en cast_df:", cast_df['movie_id'].duplicated().sum())
print("Número de IDs duplicados en crew_df:", crew_df['movie_id'].duplicated().sum())

Cast DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562474 entries, 0 to 562473
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   cast_id         562474 non-null  int64 
 1   character       562474 non-null  object
 2   credit_id       562474 non-null  object
 3   gender          562474 non-null  int64 
 4   id              562474 non-null  int64 
 5   name            562474 non-null  object
 6   order           562474 non-null  int64 
 7   profile_path    388618 non-null  object
 8   original_index  562474 non-null  int64 
 9   movie_id        562474 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 42.9+ MB
None

Crew DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464314 entries, 0 to 464313
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   credit_id       464314 non-null  object
 1   

actores

In [5]:
#Obtención de listas de nombres de Actores
df_actors = (
    cast_df.groupby('movie_id')['name'].apply(list)
    .reset_index(name='actor')
    .reset_index(drop=True)  
)

print(df_actors)


       movie_id                                              actor
0             2  [Turo Pajala, Susanna Haavisto, Matti Pellonpä...
1             3  [Matti Pellonpää, Kati Outinen, Sakari Kuosman...
2             5  [Tim Roth, Antonio Banderas, Jennifer Beals, M...
3             6  [Emilio Estevez, Cuba Gooding Jr., Denis Leary...
4            11  [Mark Hamill, Harrison Ford, Carrie Fisher, Pe...
...         ...                                                ...
43013    464207  [William Shatner, Neil deGrasse Tyson, Chris H...
43014    465044  [Karolina Antosik, Amelie Leroy, Tessa McGinn,...
43015    467731  [Lloyd Bridges, Jack Warden, Rafael Campos, Ro...
43016    468707  [Inka Haapamäki, Rosa Honkonen, Tiitus Rantala...
43017    469172  [Ruben de Freitas, Teresa Madruga, Fernando He...

[43018 rows x 2 columns]


directores

In [17]:
# Filtrado por director
df_director = crew_df[crew_df['job'] == 'Director']

# Agrupación por movie_id manteniendo sólo la primera fila (director)
df_director = df_director.groupby('movie_id').head(1)

print(len(df_director))

44545


In [7]:
df_director = df_director.rename(columns={'name': 'director'})
df_director

,credit_id,department,gender,id,job,director,profile_path,original_index,movie_id
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,0,862
109,52fe44bfc3a36847f80a7c7d,Directing,2,4945,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg,1,8844
122,52fe466a9251416c75077a89,Directing,2,26502,Director,Howard Deutch,/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg,2,15602
126,52fe44779251416c91011acb,Directing,2,2178,Director,Forest Whitaker,/4pMQkelS5lK661m9Kz3oIxLYiyS.jpg,3,31357
141,52fe44959251416c75039eef,Directing,2,56106,Director,Charles Shyer,/hnWGd74CbmTcDCFQiJ8SYLazIXW.jpg,4,11862
...,...,...,...,...,...,...,...,...,...
464291,5894a97d925141426c00818c,Directing,0,1182809,Director,Hamid Nematollah,None,45471,439050
464300,52fe4af1c3a36847f81e9b15,Directing,0,1051381,Director,Lav Diaz,/lWZ6mJCrPga6Jis1gbJYcpWpBZo.jpg,45472,111109
464306,52fe4776c3a368484e0c8387,Directing,2,67753,Director,Mark L. Lester,None,45473,67758
464311,533bccebc3a36844cf0011a7,Directing,0,1085341,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,45474,227506


In [8]:
df_directors = df_director[['movie_id', 'director']]

In [18]:
# Unión en una sola tabla
merged_df = df_directors.merge(df_actors, on='movie_id', how='inner')
print(len(merged_df))


42568


In [10]:
merged_df

,movie_id,director,actor
0,862,John Lasseter,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,Joe Johnston,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,Howard Deutch,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357,Forest Whitaker,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,Charles Shyer,"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...
42563,30840,John Irvin,"[Patrick Bergin, Uma Thurman, David Morrissey,..."
42564,439050,Hamid Nematollah,"[Leila Hatami, Kourosh Tahami, Elham Korda]"
42565,111109,Lav Diaz,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
42566,67758,Mark L. Lester,"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."


In [11]:
merged_df.to_csv(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\director_actor_df.csv")